### Stacking 堆疊法
1. 產生出m個不同演算法的base learners 模型
2. base learner 模型彼此間並無互相關聯
3. 把m個 base learner 的輸出當成新的模型的輸入，另外再訓練一個模型

### Ensemble learning 小結
Ensemble learning 有三種類型第一種為Bagging, 第二種為Boosting,第三種為Stacking

1. Bagging 採用重新採樣技巧 :  Random Forest

2. Boosting 一次產生一個新的模型，並補足前幾棵表現不好的部分:
        AdaBoost、Gradient Boosting

3. Stacking 結合許多弱學習器

In [1]:
import pandas as pd             #進行資料處理函式庫
import numpy as np              #高階大量的維度陣列與矩陣計算
import matplotlib.pyplot as plt #繪圖
import seaborn as sns           #繪圖
import io                       #負責處理資料 input/output
import requests                 #HTTP Request下載訓練資料用

### 載入資料集

In [3]:
url = "https://raw.githubusercontent.com/andy6804tw/2020-12th-ironman/master/dataset/forest/train.csv"
s = requests.get(url).content
df_data = pd.read_csv(io.StringIO(s.decode("utf-8")))
df_data = df_data.drop(labels=["Id"],axis=1)    #移除Id  axis= 1為行 0為列
df_data

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,2941,32,4,108,12,3369,219,230,147,2574,...,0,0,0,0,0,0,0,0,0,2
1,2304,20,19,350,141,1423,203,195,124,939,...,0,0,0,0,0,0,0,0,0,3
2,3397,157,13,458,46,2255,234,241,136,474,...,0,0,0,0,0,0,1,0,0,7
3,2276,122,13,470,118,1423,242,229,114,920,...,0,0,0,0,0,0,0,0,0,4
4,2780,13,8,124,4,1959,212,223,148,2652,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14359,2317,322,28,30,18,1008,138,199,195,930,...,0,0,0,0,0,0,0,0,0,6
14360,3187,337,22,488,80,3728,165,203,174,1583,...,0,0,0,0,0,0,0,1,0,2
14361,2724,36,28,175,94,1321,205,167,81,655,...,0,1,0,0,0,0,0,0,0,6
14362,3271,273,12,573,148,3621,186,243,196,1997,...,0,1,0,0,0,0,0,0,0,2


### 檢查缺失值
使用numpy 所提供的函式來檢查是否有NA缺失值，假設有缺失值使用dropna()來移除。使用的時機在於當只有少量的缺失值適用，若遇到大量缺失值的情況，或是本身的資料量就很少的情況下建議可以透過機器學習的方法補值來預測缺失值。

In [7]:
#移除缺失值
#train = train.dropna()

In [5]:
#check missing data
print("Before data clean(NAN mount):",len(np.where(np.isnan(df_data))[0]))

Before data clean(NAN mount): 0


### 資料前處理
#### 特徵標準化
通常有兩種標準化的方法:
1. min max normalization : 會將特徵數據按比例縮放到0到1區間，(或是-1到1)

2. standard deviation normalization : 會將所有特徵數據縮放成平均為0、平方差為1


#### 特徵組合
特徵需要適當地增加和減少，以提升精確度並減少計算時間

1. 增加特徵 : 特徵組合(Feature Combination)、群聚編碼(GroupBy Encoding)、產生合成樣本(Oversampling)

2. 減少特徵 : 特徵篩選(Feature Selection) 、 剔除一些樣本(Undersampling)

In [8]:
from sklearn.preprocessing import StandardScaler

def data_preprocessing(df_input):
    sc = StandardScaler()
    df = sc.fit_transform(df_input.iloc[:,0:54])
    return df

In [9]:
#取得54個特徵x
X = data_preprocessing(df_data)

In [10]:
X.shape

(14364, 54)

In [11]:
# 取得標籤y
y = df_data["Cover_Type"].values-1   #程式運作大多都從0開始，所以要-1

In [12]:
y.shape

(14364,)

### 切割訓練集與測試集

In [18]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=.3,random_state=42)

In [19]:
print("訓練集: ", X_train.shape)
print("測試集: ",X_test.shape)

訓練集:  (10054, 54)
測試集:  (4310, 54)


### XGBoost 模型
使用 XGBoost 訓練，並將結果與Stacking 做比較。

In [16]:
from xgboost import XGBClassifier

#建立XGBClassifier模型
xgboostModel = XGBClassifier()

#使用訓練資料模型
xgboostModel.fit(X_train,y_train)

#使用訓練資料預測分類
predicted = xgboostModel.predict(X_train)

d:\Anaconda\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:21:05] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [22]:
from sklearn.metrics import accuracy_score

predicted = xgboostModel.predict(X_train)
print("訓練集準確率: ",accuracy_score(y_train,predicted))

predicted = xgboostModel.predict(X_test)
print("測試集準確率: ",accuracy_score(y_test,predicted))

訓練集準確率:  0.9863735826536701
測試集準確率:  0.9846867749419954


### Stacking模型
Stacking 結合許多弱學習器，將所有的弱學習器的輸出當作新的模型的輸入接著預測最終結果

Paremeter:
1. estimatorsl : m個弱學習器
2. final_estimator : 集合所有弱學習器的輸出，訓練一個最終預測模型。預設為 LogisticRegression

Attributes :
1. estimators_ : 查看弱學習器組合
2. final_estimator : 查看最終整合訓練模型

Methods :
1. fit : 放入X、y進行模型擬合
2. predict : 預測並回傳預測類別
3. score : 預測成功的比例
4. predict_proba : 預測每個類別的機率值

In [25]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier

estimators = [
    ("rf",RandomForestClassifier()),
    ('svc',svm.SVC()),
    ("knn",KNeighborsClassifier()),
    ('dt',DecisionTreeClassifier())
]

clf = StackingClassifier(
    estimators = estimators,final_estimator = LogisticRegression()
)

clf.fit(X_train,y_train).score(X_test,y_test)

d:\Anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8545243619489559

### 測試集預測

In [26]:
from sklearn.metrics import accuracy_score

predicted = clf.predict(X_train)
print("訓練集準確率: ",accuracy_score(y_train,predicted))

predicted = clf.predict(X_test)
print("測試集準確率: ",accuracy_score(y_test,predicted))

訓練集準確率:  0.9991048338969565
測試集準確率:  0.8545243619489559
